In [13]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 4
### Team Member Names: Jacky Xu, Esha Kumar, Jingyi Fang
### Team Strategy Chosen: SAFE

In [69]:
tickers = pd.read_csv("Tickers.csv", header=None)
tickers.columns = ["ticker"]
ticker_lst = tickers['ticker'].tolist()
ticker_lst

'SHOP.TO'

In [70]:
len(ticker_lst)

62

In [71]:
def call_tickers(lst):
    called_ticker_lst = []
    for i in lst:
        called_ticker_lst.append(yf.Ticker(i))
    return called_ticker_lst


def filter_USD(lst):
    new_lst = []
    for i in lst:
        try:
            if i.info['currency'] == 'USD':
                new_lst.append(i)
        except:
            pass
    return new_lst


def filter_daily_volume(lst,start,end):
    for i in lst:
        if i.history(start = start, end = end,interval='1d')['Volume'].mean() >= 10000:
            lst.remove(i)
        return lst

In [72]:
called_tickers = call_tickers(ticker_lst)

In [73]:
usd_tickers = filter_USD(called_tickers)

In [77]:
start = "2021-07-02"
end = "2021-10-22"

final = filter_daily_volume(usd_tickers, start, end)
len(final)

55

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.